In [1]:
!pip install -q gradio --quiet
!pip install -q xformer --quiet
!pip install -q chromadb --quiet
!pip install -q langchain --quiet
!pip install -q accelerate --quiet
!pip install -q transformers --quiet
!pip install -q bitsandbytes --quiet
!pip install -q unstructured --quiet
!pip install -q sentence-transformers --quiet

^C


In [ ]:
!pip uninstall transformers
!pip install -U transformers


Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.31.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3


In [ ]:
import torch

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [ ]:
query = "Give me an indepth Recommendation System ML System Design"
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Give me an indepth Recommendation System ML System Design</b>

<p>.

I am looking for a recommendation system that can be used in a web application. I want to build a system that will recommend products to users based on their past purchases and browsing history. The system should also take into account the user's preferences, such as price range, brand, etc.

Here is my proposed design:

1. Data Collection: Collect data from various sources such as user profiles, purchase history, browsing history, ratings, reviews, etc.
2. Feature Extraction: Extract features from the collected data such as product categories, brands, prices, etc.
3. Similarity Measurement: Calculate similarities between users or products using techniques like cosine similarity, Jaccard index, etc.
4. Collaborative Filtering: Use collaborative filtering algorithms like ALS, Singular Value Decomposition (SVD), etc., to generate recommendations.
5. Content-Based Filtering: Use content-based filtering algorithms like Naive Bayes, Decision Trees, etc., to generate recommendations.
6. Hybrid Filtering: Combine both collaborative and content-based filtering to improve the accuracy of recommendations.
7. Personalization: Personalize recommendations based on user preferences, location, time of day, etc.
8. Evaluation: Evaluate the performance of the recommendation system using metrics like precision, recall, F1 score, etc.
9. Scalability: Ensure scalability by using distributed systems like Apache Spark, Hadoop, etc.
10. Security: Implement security measures to protect user data and prevent unauthorized access.

Please let me know if there are any flaws in this design or if you have any suggestions for improvement.

## Answer (1)

Your design looks good. Here are some additional points to consider:

1. **Data Quality**: Make sure your data is clean and accurate. For example, remove duplicates, handle missing values, and ensure consistency across different data sources.
2. **Feature Selection**: Select relevant features that contribute to the quality of recommendations. You may need to experiment with different feature sets to find the best combination.
3. **Model Selection**: Choose appropriate models for each stage of your pipeline. For example, use collaborative filtering for generating recommendations and content-based filtering for personalizing them.
4. **Evaluation Metrics**: Consider using a combination of evaluation metrics to assess the performance of your system. For example, use precision, recall, and F1 score for classification tasks and mean absolute error (MAE) or root mean squared error (RMSE) for regression tasks.
5. **Deployment**: Once you have built and evaluated your model, deploy it in a production environment. Consider using cloud services like AWS, Google Cloud, or Azure to host your application and scale it as needed.
6. **Monitoring and Maintenance**: Monitor the performance of your system over time and make adjustments as necessary. Keep track of errors and bugs, update your data regularly, and retrain your models periodically.</p>

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv("/content/ML system design case studies.csv")

df.head()


urls = df['Link'].to_list()

# urls = [
#     "https://www.hiberus.com/expertos-ia-generativa-ld",
#     "https://www.hiberus.com/en/experts-generative-ai-ld"
# ]


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q unstructured

In [ ]:
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

len(documents)

ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/abs/2305.14406, exception: Opening and ending tag mismatch: meta line 13 and script, line 17, column 87 (<string>, line 17)
ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/abs/2306.07415, exception: Opening and ending tag mismatch: meta line 13 and script, line 17, column 87 (<string>, line 17)
ERROR:langchain_community.document_loaders.url:Error fetching or processing https://arxiv.org/pdf/2302.01255.pdf, exception: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"


297

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_chunks = text_splitter.split_documents(documents)

len(texts_chunks)

2753

In [ ]:
documents[0]

Document(page_content='Stripe logo\n\nProducts\n\nSolutions\n\nDevelopers\n\nResources\n\nPricing\n\nContact sales\n\nSign in\n      \n      \n        Dashboard\n      \n    \n    Sign in\n\nOpen mobile navigation\n\nStripe logo\n\nBack\n\nProducts\n\nGlobal Payments\n\nAccept payments online, in person, or through your platform.\n\nRevenue and Finance Automation\n\nGrow your business with automated revenue and finance.\n\nBanking-as-a-Service\n\nEmbed financial services in your platform or product.\n\nContact Sales\n\nPricing\n\nGlobal Payments\n\nPayments\n    \n    \n  \n    \n    \n      \n        Payments\xa0\n  \n    \n      \n      \n    \n  \n\n      \n      \n      Online payments\n\nCheckout\n    \n    \n  \n    \n    \n      \n        Checkout\xa0\n  \n    \n      \n      \n    \n  \n\n      \n      \n      Prebuilt payment form\n\nElements\n    \n    \n  \n    \n    \n      \n        Elements\xa0\n  \n    \n      \n      \n    \n  \n\n      \n      \n      Customizable paym

In [ ]:
db = Chroma.from_documents(texts_chunks, embeddings, persist_directory="db")

In [ ]:
custom_template = """You are a Machine Learning System Design Interview help  AI Assistant. Given the
following conversation and a follow up question, Give an appropriate response with the ML context given to you/ '.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT,
)

In [ ]:
query = "Who you are?"
result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Who you are?</b>

<p>I am an AI language model trained to understand natural language text.</p>

In [ ]:
query = "How does Doordash improve holiday Predictions? ?"

result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>How does Doordash improve holiday Predictions? ?</b>

<p>Supervised learning involves making predictions on new data based on a labeled training set, while unsupervised learning involves finding patterns or relationships within the data itself without prior knowledge of what those patterns might represent.</p>

In [ ]:


query = "Tell me in depth - How does Improving the Performance of NLP Systems on the Gender-Neutral “They” happen at Grammarly?"

result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Tell me in depth - How does Improving the Performance of NLP Systems on the Gender-Neutral “They” happen at Grammarly?</b>

<p>Named Entity Recognition (NER) is the process of identifying and classifying named entities into predefined categories such as person, organization, location, etc. It is often used as a sub-task of Natural Language Processing (NLP) and is typically performed on unstructured text data. On the other hand, Named Entity Extraction (NEE) is the process of extracting specific named entities from a text, usually for the purpose of structured representation or analysis. It involves identifying named entities and determining their relationships to other entities in the text.</p>

In [ ]:

query = "Summarize the article - Expedia Group’s Customer Lifetime Value Prediction Model"

result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Summarize the article - Expedia Group’s Customer Lifetime Value Prediction Model</b>

<p>Named Entity Recognition (NER) is the process of identifying and classifying named entities into predefined categories such as person, organization, location, etc. It is often used as a sub-task of Natural Language Processing (NLP) and is typically performed on unstructured text data. On the other hand, Named Entity Extraction (NEE) is the process of extracting specific named entities from a text, usually for the purpose of structured representation or analysis. It involves identifying named entities and determining their relationships to other entities in the text.</p>

In [ ]:
import gradio as gr

In [ ]:
def querying(query, history):
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=db.as_retriever(search_kwargs={"k": 2}),
      memory=memory,
      condense_question_prompt=CUSTOM_QUESTION_PROMPT,
  )

  result = qa_chain({"question": query})
  return result["answer"].strip()

In [ ]:
iface = gr.ChatInterface(
    fn = querying,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="Tell me about Stripe System Design Articles?", container=False, scale=7),
    title="MLSystemDesignBot",
    theme="soft",
    examples=["How to design a System for Holiday Prediction like Doordash?",
              "Please summarize Expedia Group’s Customer Lifetime Value Prediction Model"],

    cache_examples=True,
    retry_btn="Retry",
    undo_btn="Undo",
    clear_btn="Clear",
    submit_btn="Submit"

    )

iface.launch(share=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching examples at: '/content/gradio_cached_examples/54'
Caching example 1/2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching example 2/2
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://16f489b50b4bb87b8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
